In [38]:
import numpy as np
import pandas as pd

In [45]:
reviews = pd.read_csv("yelp_dataset.txt",delimiter='\t',header=None)
reviews.columns = ["Reviews", "Reviews_class"]
reviews

,Reviews,Reviews_class
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [40]:
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
def result(df):
    Text = list()
    lines = df["Reviews"].values.tolist()
    for review in lines:
        review = review.lower()
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        review = pattern.sub('', review)
        review = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", review)
        tokens = word_tokenize(review)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        stop_words.discard("not")
        PS = PorterStemmer()
#         words = [w for w in words if not w in stop_words]
        words = [PS.stem(w) for w in words if not w in stop_words]
        words = ' '.join(words)
        Text.append(words)
    return Text

all_reviews = result(reviews)
all_reviews[0:20]

['wow love place',
 'crust not good',
 'not tasti textur nasti',
 'stop late may bank holiday rick steve recommend love',
 'select menu great price',
 'get angri want damn pho',
 'honeslti nt tast fresh',
 'potato like rubber could tell made ahead time kept warmer',
 'fri great',
 'great touch',
 'servic prompt',
 'would not go back',
 'cashier care ever say still end wayyy overpr',
 'tri cape cod ravoli chickenwith cranberrymmmm',
 'disgust pretti sure human hair',
 'shock sign indic cash',
 'highli recommend',
 'waitress littl slow servic',
 'place not worth time let alon vega',
 'not like']

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer(min_df=3)   
X = CV.fit_transform(all_reviews).toarray()

y = reviews[["Reviews_class"]].to_numpy()
print(np.shape(X))
print(np.shape(y))

(1000, 467)
(1000, 1)


In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4)
from sklearn.metrics import accuracy_score, f1_score, precision_score,recall_score

In [54]:
from xgboost import XGBClassifier
model= XGBClassifier()
model.fit(X_train,y_train)

print('Precision:',precision_score(y_test, model.predict(X_test), average='macro'))
print('Recall:',recall_score(y_test, model.predict(X_test), average='macro'))
print('F1 score:',f1_score(y_test,model.predict(X_test), average='macro'))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Precision: 0.7940170940170941
Recall: 0.7584134615384616
F1 score: 0.7557109072481094


In [56]:
from sklearn.svm import SVC,LinearSVC
svm = SVC()
svm.fit(X_train,y_train)

print('Precision:',precision_score(y_test, svm.predict(X_test), average='macro'))
print('Recall:',recall_score(y_test, svm.predict(X_test), average='macro'))
print('F1 score:',f1_score(y_test,svm.predict(X_test), average='macro'))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Precision: 0.8373294749896651
Recall: 0.8269230769230769
F1 score: 0.8279178054458953
